# 🥖 Backwaren-Preisdaten Integration

## 🎯 Ziel
Integration der Backwaren-Preisdaten aus `Preisentwicklung_Backwaren.csv` in `data_clean.csv`

## 📋 Workflow in Blöcken
1. **Setup & Imports** - Bibliotheken laden
2. **Daten laden** - Beide CSV-Dateien einlesen
3. **Datenverarbeitung** - Preisdaten aufbereiten und mergen
4. **Export** - Als neue CSV-Datei speichern
5. **Validierung** - Qualitätskontrolle

## 📁 Dateien
- **Input**: `data_clean.csv` (Hauptdaten)
- **Input**: `Preisentwicklung_Backwaren.csv` (Preisdaten)
- **Output**: `data_clean_with_bakery_prices.csv` (Ergebnis)

In [17]:
# 📦 BLOCK 1: SETUP & IMPORTS
print("📦 BLOCK 1: Setup & Imports")
print("="*50)

import pandas as pd
import numpy as np
import os
from datetime import datetime

print("✅ Bibliotheken geladen")
print("📅 Aktuelles Datum:", datetime.now().strftime("%Y-%m-%d %H:%M"))
print("📁 Arbeitsverzeichnis:", os.getcwd())
print("✅ Block 1 abgeschlossen\n")

📦 BLOCK 1: Setup & Imports
✅ Bibliotheken geladen
📅 Aktuelles Datum: 2025-06-24 16:17
📁 Arbeitsverzeichnis: /workspaces/bakery_sales_prediction/2_BaselineModel/data
✅ Block 1 abgeschlossen



In [18]:
# 📈 BLOCK 2: DATEN LADEN UND ANALYSIEREN
print("📈 BLOCK 2: Daten laden und analysieren")
print("="*50)

# Hauptdaten laden
print("📊 Lade Hauptdaten...")
df_main = pd.read_csv('data_clean.csv')
print(f"✅ Hauptdaten geladen: {df_main.shape[0]:,} Zeilen, {df_main.shape[1]} Spalten")

# Backwaren-Preisdaten laden
print("🥖 Lade Backwaren-Preisdaten...")
df_bakery = pd.read_csv('Preisentwicklung_Backwaren.csv', sep=';')
print(f"✅ Preisdaten geladen: {df_bakery.shape[0]:,} Zeilen, {df_bakery.shape[1]} Spalten")

# Datenstrukturen analysieren
print("\n🔍 Analyse der Hauptdaten:")
print(f"Datumsspalte: {df_main['Datum'].dtype}")
print(f"Datumsbereich: {df_main['Datum'].min()} bis {df_main['Datum'].max()}")

print("\n🔍 Analyse der Backwaren-Preisdaten:")
print("Verfügbare Spalten:", df_bakery.columns.tolist())
print("Erste 5 Zeilen der Preisdaten:")
print(df_bakery.head())

print("\n✅ Block 2 abgeschlossen")

📈 BLOCK 2: Daten laden und analysieren
📊 Lade Hauptdaten...
✅ Hauptdaten geladen: 9,334 Zeilen, 46 Spalten
🥖 Lade Backwaren-Preisdaten...
✅ Preisdaten geladen: 252 Zeilen, 5 Spalten

🔍 Analyse der Hauptdaten:
Datumsspalte: object
Datumsbereich: 2013-07-01 bis 2018-07-31

🔍 Analyse der Backwaren-Preisdaten:
Verfügbare Spalten: ['time', '1_variable_attribute_code', '1_variable_attribute_label', 'value', 'value_unit']
Erste 5 Zeilen der Preisdaten:
   time 1_variable_attribute_code 1_variable_attribute_label value value_unit
0  2013                   MONAT01                     Januar  91,6   2020=100
1  2013                   MONAT01                     Januar    88   2020=100
2  2013                   MONAT01                     Januar  88,8   2020=100
3  2013                   MONAT02                    Februar  91,9   2020=100
4  2013                   MONAT02                    Februar    88   2020=100

✅ Block 2 abgeschlossen


In [19]:
# ⚙️ BLOCK 3: DATENVERARBEITUNG - PREISDATEN INTEGRIEREN
print("⚙️ BLOCK 3: Datenverarbeitung")
print("="*50)

# 1. Backwaren-Preisdaten bereits in Block 2 geladen als df_bakery
print("🥖 Verwende bereits geladene Backwaren-Preisdaten...")
print(f"✅ Preisdaten verfügbar: {df_bakery.shape[0]:,} Zeilen")

# 2. Alle Preisdaten verwenden (da nur eine Kategorie vorhanden)
print("🔍 Verwende alle verfügbaren Preisdaten...")
bakery_clean = df_bakery.copy()
print(f"✅ Alle Daten: {len(bakery_clean)} Einträge")

# 3. Zeitstempel und Werte aufbereiten
print("📅 Bereite Zeitstempel auf...")

# Monat-Mapping
month_map = {f'MONAT{i:02d}': i for i in range(1, 13)}

# Zeitstempel erstellen
bakery_clean['Jahr'] = bakery_clean['time']
bakery_clean['Monat'] = bakery_clean['1_variable_attribute_code'].map(month_map)

# Value-Spalte bereinigen (Komma zu Punkt)
bakery_clean['VPI_Backwaren'] = pd.to_numeric(
    bakery_clean['value'].astype(str).str.replace(',', '.'), 
    errors='coerce'
)

# Nur relevante Spalten behalten und nach Jahr/Monat aggregieren (Mittelwert bei mehreren Werten pro Monat)
price_data = bakery_clean[['Jahr', 'Monat', 'VPI_Backwaren']].dropna()
price_data = price_data.groupby(['Jahr', 'Monat'])['VPI_Backwaren'].mean().reset_index()
price_data = price_data.sort_values(['Jahr', 'Monat']).reset_index(drop=True)

print(f"✅ Preisdaten aufbereitet: {len(price_data)} Jahr-Monat-Kombinationen")
print(f"📊 Zeitraum Preisdaten: {price_data['Jahr'].min():.0f}-{price_data['Jahr'].max():.0f}")

# 4. Hauptdaten vorbereiten
print("📊 Bereite Hauptdaten vor...")
df_main['Datum'] = pd.to_datetime(df_main['Datum'])
df_main['Jahr'] = df_main['Datum'].dt.year
df_main['Monat'] = df_main['Datum'].dt.month

# Zeitraum der Hauptdaten
main_start = df_main['Jahr'].min()
main_end = df_main['Jahr'].max()
print(f"📅 Zeitraum Hauptdaten: {main_start}-{main_end}")

# 5. Merge durchführen
print("🔗 Führe Merge durch...")
df_merged = pd.merge(
    df_main, 
    price_data, 
    on=['Jahr', 'Monat'], 
    how='left'
)

print(f"✅ Nach Merge: {len(df_merged)} Zeilen")

# 6. Forward Fill für fehlende Werte
print("📈 Forward Fill für fehlende Preisdaten...")
missing_before = df_merged['VPI_Backwaren'].isnull().sum()
df_merged['VPI_Backwaren'] = df_merged['VPI_Backwaren'].fillna(method='ffill')
missing_after = df_merged['VPI_Backwaren'].isnull().sum()

print(f"📊 Fehlende Werte: {missing_before} → {missing_after}")

# 7. Daten außerhalb des Preis-Zeitraums anzeigen
price_start = price_data['Jahr'].min()
price_end = price_data['Jahr'].max()

if price_start > main_start or price_end < main_end:
    print(f"⚠️ Zeitraum-Unterschied erkannt:")
    print(f"   Preisdaten: {price_start:.0f}-{price_end:.0f}")
    print(f"   Hauptdaten: {main_start}-{main_end}")

# 8. Temporäre Spalten entfernen
df_final = df_merged.drop(columns=['Jahr', 'Monat'])

print(f"✅ Finale Daten: {len(df_final)} Zeilen, {len(df_final.columns)} Spalten")
print(f"📈 Neue Spalte hinzugefügt: VPI_Backwaren")

# Kurze Statistik der neuen Spalte
if 'VPI_Backwaren' in df_final.columns:
    vpi_stats = df_final['VPI_Backwaren'].describe()
    print(f"📊 VPI_Backwaren Statistik:")
    print(f"   Min: {vpi_stats['min']:.2f}")
    print(f"   Max: {vpi_stats['max']:.2f}")
    print(f"   Median: {vpi_stats['50%']:.2f}")

print("\n✅ Block 3 abgeschlossen")

⚙️ BLOCK 3: Datenverarbeitung
🥖 Verwende bereits geladene Backwaren-Preisdaten...
✅ Preisdaten verfügbar: 252 Zeilen
🔍 Verwende alle verfügbaren Preisdaten...
✅ Alle Daten: 252 Einträge
📅 Bereite Zeitstempel auf...
✅ Preisdaten aufbereitet: 84 Jahr-Monat-Kombinationen
📊 Zeitraum Preisdaten: 2013-2019
📊 Bereite Hauptdaten vor...
📅 Zeitraum Hauptdaten: 2013-2018
🔗 Führe Merge durch...
✅ Nach Merge: 9334 Zeilen
📈 Forward Fill für fehlende Preisdaten...
📊 Fehlende Werte: 0 → 0
✅ Finale Daten: 9334 Zeilen, 46 Spalten
📈 Neue Spalte hinzugefügt: VPI_Backwaren
📊 VPI_Backwaren Statistik:
   Min: 90.33
   Max: 96.83
   Median: 92.63

✅ Block 3 abgeschlossen


/tmp/ipykernel_6543/1219488465.py:63: FutureWarning: Series.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  df_merged['VPI_Backwaren'] = df_merged['VPI_Backwaren'].fillna(method='ffill')


In [20]:
# 🗑️ BLOCK 4: DATENBEREINIGUNG - FEIERTAGS-SPALTEN ENTFERNEN
print("🗑️ BLOCK 4: Datenbereinigung")
print("="*50)

# Liste der zu entfernenden Feiertags-Spalten
columns_to_remove = [
    'Feiertag_Ostermontag',
    'Feiertag_Ostern', 
    'Feiertag_Pfingsten',
    'Feiertag_Pfingstmontag',
    'Feiertag_Silvester',
    'Feiertag_Tag der Arbeit'
]

print("🔍 Prüfe vorhandene Feiertags-Spalten...")
print(f"📊 Spalten vor Bereinigung: {len(df_final.columns)}")

# Prüfen welche Spalten tatsächlich vorhanden sind
existing_columns = [col for col in columns_to_remove if col in df_final.columns]
missing_columns = [col for col in columns_to_remove if col not in df_final.columns]

if existing_columns:
    print(f"✅ Gefundene Spalten zum Entfernen: {len(existing_columns)}")
    for col in existing_columns:
        print(f"   • {col}")
    
    # Spalten entfernen
    df_final = df_final.drop(columns=existing_columns)
    print(f"🗑️ {len(existing_columns)} Feiertags-Spalten entfernt")
else:
    print("ℹ️ Keine der angegebenen Feiertags-Spalten gefunden")

if missing_columns:
    print(f"⚠️ Nicht gefundene Spalten: {len(missing_columns)}")
    for col in missing_columns:
        print(f"   • {col}")

print(f"📊 Spalten nach Bereinigung: {len(df_final.columns)}")
print(f"📉 Reduzierung: {len(existing_columns)} Spalten entfernt")

print("\n✅ Block 4 abgeschlossen")

🗑️ BLOCK 4: Datenbereinigung
🔍 Prüfe vorhandene Feiertags-Spalten...
📊 Spalten vor Bereinigung: 46
✅ Gefundene Spalten zum Entfernen: 5
   • Feiertag_Ostern
   • Feiertag_Pfingsten
   • Feiertag_Pfingstmontag
   • Feiertag_Silvester
   • Feiertag_Tag der Arbeit
🗑️ 5 Feiertags-Spalten entfernt
⚠️ Nicht gefundene Spalten: 1
   • Feiertag_Ostermontag
📊 Spalten nach Bereinigung: 41
📉 Reduzierung: 5 Spalten entfernt

✅ Block 4 abgeschlossen


In [21]:
# 🌦️ BLOCK 4.5: WETTERCODE-BEREINIGUNG MIT K-NEAREST IMPUTATION
print("🌦️ BLOCK 4.5: Wettercode-Bereinigung")
print("="*50)

from sklearn.impute import KNNImputer

# Wettercode-Spalten identifizieren
weather_cols = [col for col in df_final.columns if 'wettercode' in col.lower() or 'weather' in col.lower()]
print(f"🔍 Gefundene Wettercode-Spalten: {len(weather_cols)}")
for col in weather_cols:
    print(f"   • {col}")

if weather_cols:
    for col in weather_cols:
        print(f"\n📊 Verarbeite Spalte: {col}")
        
        # Anzahl der "99"-Werte prüfen
        count_99 = (df_final[col] == 99).sum()
        total_values = len(df_final[col])
        percentage_99 = (count_99 / total_values) * 100
        
        print(f"   • Werte '99': {count_99} ({percentage_99:.1f}%)")
        print(f"   • Gültige Werte: {total_values - count_99}")
        
        if count_99 > 0:
            # "99"-Werte als NaN markieren
            df_final[col] = df_final[col].replace(99, np.nan)
            
            # Bereite Features für KNN-Imputation vor
            # Verwende numerische Spalten als Features (ohne Datum und Target)
            feature_cols = df_final.select_dtypes(include=[np.number]).columns.tolist()
            feature_cols = [c for c in feature_cols if c != col]  # Target-Spalte ausschließen
            
            # Erstelle DataFrame für Imputation
            impute_data = df_final[[col] + feature_cols].copy()
            
            # KNN-Imputation durchführen
            print(f"   🔧 Führe k-nearest Imputation durch...")
            knn_imputer = KNNImputer(n_neighbors=5, weights='uniform')
            
            try:
                # Imputation anwenden
                imputed_values = knn_imputer.fit_transform(impute_data)
                
                # Nur die Target-Spalte zurückschreiben
                df_final[col] = imputed_values[:, 0]
                
                # Auf ganze Zahlen runden (da Wettercode diskret ist)
                df_final[col] = np.round(df_final[col]).astype(int)
                
                # Validierung nach Imputation
                remaining_nan = df_final[col].isnull().sum()
                print(f"   ✅ Imputation abgeschlossen")
                print(f"   • Verbleibende NaN: {remaining_nan}")
                
                # Werteverteilung nach Imputation
                unique_values = sorted(df_final[col].unique())
                print(f"   • Eindeutige Werte nach Imputation: {unique_values[:10]}...")  # Erste 10 zeigen
                
            except Exception as e:
                print(f"   ❌ Fehler bei Imputation: {e}")
                print(f"   🔄 Verwende Forward-Fill als Fallback...")
                df_final[col] = df_final[col].fillna(method='ffill').fillna(method='bfill')
        else:
            print(f"   ✅ Keine '99'-Werte gefunden")
else:
    print("ℹ️ Keine Wettercode-Spalten gefunden")
    # Suche nach anderen möglichen Wetterspalten
    potential_weather = [col for col in df_final.columns if any(word in col.lower() for word in ['temp', 'rain', 'wind', 'humid', 'press'])]
    if potential_weather:
        print(f"💡 Mögliche Wetter-Spalten gefunden: {potential_weather}")

# Zusammenfassung
print(f"\n📋 Zusammenfassung Wettercode-Bereinigung:")
print(f"   • Verarbeitete Spalten: {len(weather_cols)}")
print(f"   • Methode: k-nearest neighbors (k=5)")
print(f"   • Fallback: Forward/Backward Fill")

print("\n✅ Block 4.5 abgeschlossen")

🌦️ BLOCK 4.5: Wettercode-Bereinigung
🔍 Gefundene Wettercode-Spalten: 2
   • Wettercode
   • Wettercode_fehlt

📊 Verarbeite Spalte: Wettercode
   • Werte '99': 2325 (24.9%)
   • Gültige Werte: 7009
   🔧 Führe k-nearest Imputation durch...


   ✅ Imputation abgeschlossen
   • Verbleibende NaN: 0
   • Eindeutige Werte nach Imputation: [np.int64(0), np.int64(1), np.int64(2), np.int64(3), np.int64(4), np.int64(5), np.int64(6), np.int64(7), np.int64(8), np.int64(9)]...

📊 Verarbeite Spalte: Wettercode_fehlt
   • Werte '99': 0 (0.0%)
   • Gültige Werte: 9334
   ✅ Keine '99'-Werte gefunden

📋 Zusammenfassung Wettercode-Bereinigung:
   • Verarbeitete Spalten: 2
   • Methode: k-nearest neighbors (k=5)
   • Fallback: Forward/Backward Fill

✅ Block 4.5 abgeschlossen


In [22]:
# 💾 BLOCK 5: EXPORT - FINALE DATEN SPEICHERN
print("💾 BLOCK 5: Export")
print("="*50)

# Dateiname für Export definieren
output_filename = 'data_clean_with_bakery_sales.csv'

print(f"📁 Speichere finale Daten als: {output_filename}")

try:
    # CSV-Export durchführen
    df_final.to_csv(output_filename, index=False)
    
    # Dateigröße prüfen
    file_size = os.path.getsize(output_filename)
    file_size_mb = file_size / (1024 * 1024)
    
    print(f"✅ Export erfolgreich!")
    print(f"📊 Finale Datei-Statistiken:")
    print(f"   • Dateiname: {output_filename}")
    print(f"   • Dateigröße: {file_size:,} Bytes ({file_size_mb:.2f} MB)")
    print(f"   • Zeilen: {len(df_final):,}")
    print(f"   • Spalten: {len(df_final.columns)}")
    
    # Spalten-Übersicht
    print(f"\n📋 Spalten in der finalen Datei:")
    for i, col in enumerate(df_final.columns, 1):
        marker = "🆕" if col == "VPI_Backwaren" else "📊"
        print(f"   {i:2d}. {marker} {col}")
    
    # Datenvorschau
    print(f"\n👀 Datenvorschau (erste 3 Zeilen):")
    preview_cols = ['Datum', 'Umsatz', 'VPI_Backwaren'] if 'VPI_Backwaren' in df_final.columns else ['Datum', 'Umsatz']
    available_preview_cols = [col for col in preview_cols if col in df_final.columns]
    
    if available_preview_cols:
        print(df_final[available_preview_cols].head(3).to_string(index=False))
    
    # Zeitraum-Info
    if 'Datum' in df_final.columns:
        print(f"\n📅 Zeitraum der finalen Daten:")
        print(f"   • Von: {df_final['Datum'].min()}")
        print(f"   • Bis: {df_final['Datum'].max()}")
        print(f"   • Tage: {len(df_final)} Datensätze")

except Exception as e:
    print(f"❌ Fehler beim Export: {e}")
    print("💡 Mögliche Lösungen:")
    print("   • Schreibrechte im Ordner prüfen")
    print("   • Ausreichend Speicherplatz verfügbar?")
    print("   • Datei bereits geöffnet in Excel?")

print("\n✅ Block 5 abgeschlossen")
print("\n🎉 GESAMTER WORKFLOW ABGESCHLOSSEN!")
print("="*50)

💾 BLOCK 5: Export
📁 Speichere finale Daten als: data_clean_with_bakery_sales.csv
✅ Export erfolgreich!
📊 Finale Datei-Statistiken:
   • Dateiname: data_clean_with_bakery_sales.csv
   • Dateigröße: 1,271,464 Bytes (1.21 MB)
   • Zeilen: 9,334
   • Spalten: 41

📋 Spalten in der finalen Datei:
    1. 📊 id
    2. 📊 Datum
    3. 📊 Umsatz
    4. 📊 KielerWoche
    5. 📊 Bewoelkung
    6. 📊 Temperatur
    7. 📊 Windgeschwindigkeit
    8. 📊 Wettercode
    9. 📊 ist_feiertag
   10. 📊 feiertag_vortag
   11. 📊 feiertag_folgetag
   12. 📊 Wettercode_fehlt
   13. 📊 Warengruppe_Brot
   14. 📊 Warengruppe_Brötchen
   15. 📊 Warengruppe_Croissant
   16. 📊 Warengruppe_Konditorei
   17. 📊 Warengruppe_Kuchen
   18. 📊 Warengruppe_Saisonbrot
   19. 📊 Wochentag_Monday
   20. 📊 Wochentag_Saturday
   21. 📊 Wochentag_Sunday
   22. 📊 Wochentag_Thursday
   23. 📊 Wochentag_Tuesday
   24. 📊 Wochentag_Wednesday
   25. 📊 Monat_2
   26. 📊 Monat_3
   27. 📊 Monat_4
   28. 📊 Monat_5
   29. 📊 Monat_6
   30. 📊 Monat_7
   31. 📊 M